<img src="saverlife.png" width="400">

# Chat Genie Demo
### Write a question and hit enter

In [11]:
import ipywidgets as widgets
from fastai.text.all import *
import urllib.request
from os.path import exists
import requests

In [12]:
def _end_headline(x):
    if type(x) != str: return ''
    if x[-1] in ['.', '?', '!', '…', ':', '“', '”']: return x + ' '
    else: return x + '. '

def breakup_text(text, max=90):
    if len(text) < max:
        print(text)
    else:
        for i in range(max-1,1,-1):
            if text[i] == ' ':
                print(text[:i])
                break
        breakup_text(text[i:].strip())

In [13]:
# MODEL_URL = "https://www.dropbox.com/s/tmu82r4pd9h0ek8/chat_model.pkl?dl=1"
# if not exists('model.pkl'):
#     urllib.request.urlretrieve(MODEL_URL, "model.pkl")

# learn = load_learner("model.pkl")
learn = load_learner("seg_learner_may2.pkl")

sdf = pd.read_pickle('segment_df_may2.pkl')

In [14]:
text = 'how do i improve credit'

In [15]:
pred = learn.predict(text)
df = pd.DataFrame(pred[2], columns = ['pct'])
df['label_idx'] = df.index
df = df.sort_values('pct', ascending=False).iloc[:50]
url_df = pd.DataFrame(learn.dls.vocab[1])
url_df['label_idx'] = url_df.index
df = df.merge(url_df)
df.columns = ['pct', 'idx', 'url_segment']
df = df.merge(sdf)
df['score'] = df['pct'] / (df['text length'] + 30) * 10000
df = df.sort_values('score', ascending=False)

In [28]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

class preds:
    DF = None
    QUESTION = None
    NUM = None
    HEADLINE = None
    SUB = None
    
def predictions(text):
    pred = learn.predict(text)
    df = pd.DataFrame(pred[2], columns = ['pct'])
    df['label_idx'] = df.index
    df = df.sort_values('pct', ascending=False).iloc[:50]
    url_df = pd.DataFrame(learn.dls.vocab[1])
    url_df['label_idx'] = url_df.index
    df = df.merge(url_df)
    df.columns = ['pct', 'idx', 'url_segment']
    df = df.merge(sdf)
    df['score'] = df['pct'] / (df['text length'] + 20) * 10000
    rows = (df.top_tag == 'Stories') | (df.top_tag == 'Saver Stories')
    df.loc[rows, 'score'] = df.loc[rows, 'score'] / 5
    df = df.sort_values('score', ascending=False)
    preds.DF = df
    return(df)

def chat_response(text, i):
    if i == 0:
        df = predictions(text)
        for rate in [rate1, rate2, rate3, rate4, rate5,
                    rate6, rate7, rate8, rate9, rate10]:
            rate.value = 'Did that answer your question?'
    else:
        df = preds.DF
    print(color.BOLD + color.UNDERLINE)
    preds.HEADLINE = df.iloc[i]['headline']
    breakup_text(df.iloc[i]['headline'])
    print(color.END + color.BOLD + color.DARKCYAN)
    preds.SUB = df.iloc[i]['subheading']
    breakup_text(df.iloc[i]['subheading'])
    print(color.END)
    breakup_text(df.iloc[i]['text'])
    print(f"\n{color.GREEN}Read the full article on the SaverLife Blog")
    print(df.iloc[i]['url'])
    print(color.END)
    
    
def chat1(text): chat_response(text, 0)
def chat2(text): chat_response(text, 1)
def chat3(text): chat_response(text, 2)
def chat4(text): chat_response(text, 3)
def chat5(text): chat_response(text, 4)
def chat6(text): chat_response(text, 5)
def chat7(text): chat_response(text, 6)
def chat8(text): chat_response(text, 7)
def chat9(text): chat_response(text, 8)
def chat10(text): chat_response(text, 9)
    
def save_rating(question, rating, i):
    if rating != 'Did that answer your question?':
        question = question.replace("'", "''")
        dic = {'Yes': 1, 'Somewhat': 2, 'Not at all': 3}
        rating = dic[rating]
        url = preds.DF.iloc[i]['url']
        segment = preds.DF.iloc[i]['segment']
        headers = {'Content-type': 'application/json'}
        data = '{\"question\":\"' + question + \
            '\",\"heading\":\"' + url + \
            '\",\"subheading\":\"' + str(int(segment)) + \
            '\",\"rating\":' + str(rating) + '}'
        response = requests.post(
            'https://sl2-datascience-api.herokuapp.com/rate_bot_response/',
            headers=headers, data=data)
        if response.json():
            print('There was an error storing your rating. Did you use "quotes"?')
            print(response.json())
        else: print('Thank You for your rating')
        response.close()
    else: print('')
    
def save_rating_1(rating, question): save_rating(question, rating, 0)
def save_rating_2(rating, question): save_rating(question, rating, 1)
def save_rating_3(rating, question): save_rating(question, rating, 2)
def save_rating_4(rating, question): save_rating(question, rating, 3)
def save_rating_5(rating, question): save_rating(question, rating, 4)
def save_rating_6(rating, question): save_rating(question, rating, 5)
def save_rating_7(rating, question): save_rating(question, rating, 6)
def save_rating_8(rating, question): save_rating(question, rating, 7)
def save_rating_9(rating, question): save_rating(question, rating, 8)
def save_rating_10(rating, question): save_rating(question, rating, 9)

In [17]:
textbox = widgets.Text(
    value='How do I save more money?',
    placeholder='Write your question here!',
    description='Question:',
    disabled=False,
    continuous_update=False,
    layout=widgets.Layout(width='500px')
)

rate1 = widgets.Dropdown(
    options=['Did that answer your question?', 'Yes', 'Somewhat', "Not at all"],
    value='Did that answer your question?',
    description='Rating #1', disabled=False)

rate2 = widgets.Dropdown(
    options=['Did that answer your question?', 'Yes', 'Somewhat', "Not at all"],
    value='Did that answer your question?',
    description='Rating #2', disabled=False)

rate3 = widgets.Dropdown(
    options=['Did that answer your question?', 'Yes', 'Somewhat', "Not at all"],
    value='Did that answer your question?',
    description='Rating #3', disabled=False)

rate4 = widgets.Dropdown(
    options=['Did that answer your question?', 'Yes', 'Somewhat', "Not at all"],
    value='Did that answer your question?',
    description='Rating #4', disabled=False)

rate5 = widgets.Dropdown(
    options=['Did that answer your question?', 'Yes', 'Somewhat', "Not at all"],
    value='Did that answer your question?',
    description='Rating #5', disabled=False)

rate6 = widgets.Dropdown(
    options=['Did that answer your question?', 'Yes', 'Somewhat', "Not at all"],
    value='Did that answer your question?',
    description='Rating #6', disabled=False)

rate7 = widgets.Dropdown(
    options=['Did that answer your question?', 'Yes', 'Somewhat', "Not at all"],
    value='Did that answer your question?',
    description='Rating #7', disabled=False)

rate8 = widgets.Dropdown(
    options=['Did that answer your question?', 'Yes', 'Somewhat', "Not at all"],
    value='Did that answer your question?',
    description='Rating #8', disabled=False)

rate9 = widgets.Dropdown(
    options=['Did that answer your question?', 'Yes', 'Somewhat', "Not at all"],
    value='Did that answer your question?',
    description='Rating #9', disabled=False)

rate10 = widgets.Dropdown(
    options=['Did that answer your question?', 'Yes', 'Somewhat', "Not at all"],
    value='Did that answer your question?',
    description='Rating #10', disabled=False)

out1 = widgets.interactive_output(chat1, {'text': textbox})
out2 = widgets.interactive_output(chat2, {'text': textbox})
out3 = widgets.interactive_output(chat3, {'text': textbox})
out4 = widgets.interactive_output(chat4, {'text': textbox})
out5 = widgets.interactive_output(chat5, {'text': textbox})
out6 = widgets.interactive_output(chat6, {'text': textbox})
out7 = widgets.interactive_output(chat7, {'text': textbox})
out8 = widgets.interactive_output(chat8, {'text': textbox})
out9 = widgets.interactive_output(chat9, {'text': textbox})
out10 = widgets.interactive_output(chat10, {'text': textbox})

save1 = widgets.interactive_output(save_rating_1, {'rating': rate1, 'question': textbox})
save2 = widgets.interactive_output(save_rating_2, {'rating': rate2, 'question': textbox})
save3 = widgets.interactive_output(save_rating_3, {'rating': rate3, 'question': textbox})
save4 = widgets.interactive_output(save_rating_4, {'rating': rate4, 'question': textbox})
save5 = widgets.interactive_output(save_rating_5, {'rating': rate5, 'question': textbox})
save6 = widgets.interactive_output(save_rating_1, {'rating': rate6, 'question': textbox})
save7 = widgets.interactive_output(save_rating_2, {'rating': rate7, 'question': textbox})
save8 = widgets.interactive_output(save_rating_3, {'rating': rate8, 'question': textbox})
save9 = widgets.interactive_output(save_rating_4, {'rating': rate9, 'question': textbox})
save10 = widgets.interactive_output(save_rating_5, {'rating': rate10, 'question': textbox})

In [18]:
widgets.HBox([widgets.VBox([textbox])])

### Top 10 Responses

In [19]:
widgets.HBox([widgets.VBox([out1, rate1, save1, out2, rate2, save2, out3, rate3, save3])])

In [20]:
widgets.HBox([widgets.VBox([out4, rate4, save4, out5, rate5, save5, out6, rate6, save6])])

In [21]:
widgets.HBox([widgets.VBox([out7, rate7, save7, out8, rate8, save8, out9, rate9, save9, out10, rate10, save10])])